In [ ]:
import os
import subprocess
import requests
import json
import sys
import pandas as pd

import ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context





#from tcga downloader import *
#ids=process_manifests(manifest_file)

'''
Match tumor and normal samples to a patient through GDC API. This will provide metadata that
for associated files including sample type(tumor /normal) patient ID(TCGA barcode), etc
The general query structure will look like this:
curl "https://gdc-api.nci.nih.gov/files?size=100000&format=tsv&filters=XXXX&fields=YYYY"
filters= :
The XXXX will be replaced with a URL-encoded JSON query that will filter a list of files. The JSON query can be generated yourself and then URL encoded, or the filtering can be performed on the GDC Data Portal (https://gdc-portal.nci.nih.gov/search/s?facetTab=files). You can copy and paste the string that appears where the * are displayed in the following:
https://gdc-portal.nci.nih.gov/search/f?filters=**********&facetTab=cases
'''

def prepare_payload(ids,data_type=None):

    #"Gene Expression Quantification"
    
    "Workflow Type" "HTSeq-Counts"

    "Data Category" "transcriptome profilling"
    "Experimental Strategy" "RNA-Seq"

    no_of_samples=len(ids)
    part1='''{
    "filters":{
        "op":"and",
        "content":[
            {
                "op":"in",
                "content":{
                    "field":"files.file_id",
                    "value":[%s]
                }
            },
            {
                "op":"=",
                "content":{
                    "field":"files.data_type",
                    "value":"%s"
                }
            }
        ]
    },'''%(",\n".join(ids),data_type)


    part2=''' "format":"TSV","fields":"file_id,file_name,cases.submitter_id,cases.disease_type,cases.case_id,data_category,data_type,cases.samples.tumor_descriptor,cases.samples.tissue_type,cases.samples.sample_type,cases.samples.submitter_id,cases.samples.sample_id,cases.samples.portions.analytes.aliquots.aliquot_id,cases.samples.portions.analytes.aliquots.submitter_id","size":"%d"}'''%no_of_samples

    payload_command='%s %s'%(part1,part2)

    payloadfile='payloadv3.txt'
    with open(payloadfile,'w') as output_:
        output_.write(payload_command)
    return payloadfile


def get_ids(manifest):
    try:
#manifest='all.txt'
        with open (manifest,'r') as input_:
            ids=["\"%s\""%i.strip('\n').split('\t')[0] for i in input_][1:]

        no_of_samples=len(ids)
        return ids
    except Exception as ex:
        return None
    #print(ids)


def get_metadata(payloadfile):
    metadata='Metadata.tsv'
    webaddr='\'https://api.gdc.cancer.gov/files\''
    args=['curl', '--request POST', '--header','\"Content-Type: application/json\"',
      '--data','@%s'%payloadfile,webaddr, '>', metadata]

    single=' '.join(args)
    print(single)
    os.system(single)
    return metadata


#def get_metadatada():
 #   ids=process_manifest()
  #  if ids==None:
   #     print('Error encountered\nPlease ensure that you are using the correct manifest file')
   # else:
   #     payloadfile=prepare_payload(ids)
   #     download_data(payloadfile)

def download_data(metadatafile,sep='\t',outdir='downloads'):
    df=pd.DataFrame()
    data_df=pd.read_csv(metadatafile,sep=sep)
    sampletypes=data_df['cases.0.samples.0.sample_type'].unique()
    sampletypes=sampletypes.tolist()
    curr_dir=os.getcwd()
    
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    else:
        print('output directory exists\ndata may be overwritten')
        
    for sampletype in sampletypes:
        sel=data_df[data_df['cases.0.samples.0.sample_type'].str.contains(sampletype)][['file_id',
                                                                               'file_name']]

        sampledir="%s/%s"%(outdir,sampletype)
        
        if not os.path.exists(sampledir):
            os.mkdir(sampledir)
        else:
            print('sample type directory exists\ndata may be overwritten')
            
        
        os.chdir(sampledir)
        all_file_ids=sel['file_id'].values
        download_list=[]
        for file_id in all_file_ids:
            args=['curl', '--remote-name', '--remote-header-name',
                  '\'https://api.gdc.cancer.gov/data/%s\''%file_id]
            cmd=' '.join(args)
            print('downloading %s'%file_id)
            os.system(cmd)
            #download_list.append(' '.join(args))
        os.chdir(curr_dir)
    print('Download complete\nAll data has been downloaded to ------------->%s'%outdir)
        
        
    

In [8]:
def splitAndUnzipFiles(inputDirNormal, inputDirNotNormal, outputDir):

    import os, zipfile
    import gzip
    import shutil
    import csv
    extension = ".gz"
    with open("Metadata.tsv") as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"') 
    #normal files
    for item in os.listdir(inputDirNormal): 
        if item.endswith(extension): # check for ".zip" extension
            file_name = inputDirNormal + "/" + item
            name = item.rsplit(".",1)
            with open("Metadata.tsv") as fd:
                rd = csv.reader(fd, delimiter="\t", quotechar='"') 
                for row in rd:
                    print(row[13],item)
                    if row[13] == item:
                        break
            out_file = outputDir + "/" + "normal"+"_"+row[14]
            print(out_file)
            with gzip.open(file_name, 'rb') as f_in:
                with open(out_file, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

    #not normal files  
    for item in os.listdir(inputDirNotNormal): 
        if item.endswith(extension): # check for ".zip" extension
            file_name = inputDirNotNormal + "/" + item
            name = item.rsplit(".",1)
            with open("Metadata.tsv") as fd:
                rd = csv.reader(fd, delimiter="\t", quotechar='"') 
                found = False
                for row in rd:
                    #print(row[13],item)
                    if row[13] == item:
                        found = True
                        break
                if found == False: print("NOT FOUND")    
            out_file = outputDir + "/" + "notnormal"+"_"+row[14]
            #print(out_file)
            with gzip.open(file_name, 'rb') as f_in:
                with open(out_file, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

In [ ]:
#sample code
#download the manifest file from TGCA
ids = get_ids("TGCA-COAD/tgca_coad_manifest.txt") 
print(len(ids))
payload = prepare_payload(ids, data_type = 'Gene Expression Quantification')
metadata = get_metadata(payload)
download_data(metadata,sep='\t',outdir='TGCA-COAD')


In [9]:
splitAndUnzipFiles('TGCA-COAD/Solid Tissue Normal','TGCA-COAD/Primary Tumor','TGCA-COAD/extracted')

file_name 9e8b528b-1172-4c07-a09b-ebb23cf2310c.htseq.counts.gz
1cbb39ca-efbf-4080-bd32-1b76006394a8.htseq.counts.gz 9e8b528b-1172-4c07-a09b-ebb23cf2310c.htseq.counts.gz
43b292be-5d63-4523-a43f-666d20039208.htseq.counts.gz 9e8b528b-1172-4c07-a09b-ebb23cf2310c.htseq.counts.gz
df959336-a712-47a9-9042-b2b6bc01e813.htseq.counts.gz 9e8b528b-1172-4c07-a09b-ebb23cf2310c.htseq.counts.gz
b57f69a6-91ca-4131-a654-5495f6dd1131.htseq.counts.gz 9e8b528b-1172-4c07-a09b-ebb23cf2310c.htseq.counts.gz
bf9c448b-bdc9-4f74-b13a-374e6add7939.htseq.counts.gz 9e8b528b-1172-4c07-a09b-ebb23cf2310c.htseq.counts.gz
2c1aca0d-6c50-4f1e-9edf-fec9ec32da69.htseq.counts.gz 9e8b528b-1172-4c07-a09b-ebb23cf2310c.htseq.counts.gz
bd8839a3-4dcb-4688-b051-6f09c18d7fc7.htseq.counts.gz 9e8b528b-1172-4c07-a09b-ebb23cf2310c.htseq.counts.gz
65ec87f8-ca6b-4b47-9700-ef1b67589dec.htseq.counts.gz 9e8b528b-1172-4c07-a09b-ebb23cf2310c.htseq.counts.gz
bda1a9a4-a14f-4463-81d2-a4fcca65d6f1.htseq.counts.gz 9e8b528b-1172-4c07-a09b-ebb23cf2310c

b1684b0e-0c14-434a-94b0-78c42adf4ea9.htseq.counts.gz 09f2bb20-909e-4929-8a79-c64c7fb2af31.htseq.counts.gz
0ee894a2-b2d9-4d06-bba4-dc1e915c698a.htseq.counts.gz 09f2bb20-909e-4929-8a79-c64c7fb2af31.htseq.counts.gz
29dab10a-a0a0-4dbd-b574-97c22b70a3f1.htseq.counts.gz 09f2bb20-909e-4929-8a79-c64c7fb2af31.htseq.counts.gz
2f906887-7836-4f24-8f97-91b10811a7c3.htseq.counts.gz 09f2bb20-909e-4929-8a79-c64c7fb2af31.htseq.counts.gz
978145ba-cca3-4244-8a98-9cedb455bd9b.htseq.counts.gz 09f2bb20-909e-4929-8a79-c64c7fb2af31.htseq.counts.gz
5faf3586-b033-45fd-8dd9-fef111b0f752.htseq.counts.gz 09f2bb20-909e-4929-8a79-c64c7fb2af31.htseq.counts.gz
09f2bb20-909e-4929-8a79-c64c7fb2af31.htseq.counts.gz 09f2bb20-909e-4929-8a79-c64c7fb2af31.htseq.counts.gz
TGCA-COAD/extracted/normal_2c3b3931-b635-46c6-a527-5293378ab196
file_name f1fcd165-ddbc-44b8-b062-ae5199c53cba.htseq.counts.gz
1cbb39ca-efbf-4080-bd32-1b76006394a8.htseq.counts.gz f1fcd165-ddbc-44b8-b062-ae5199c53cba.htseq.counts.gz
43b292be-5d63-4523-a43f-6

file_name c789f723-e543-4b62-98b4-444e5079fa71.htseq.counts.gz
1cbb39ca-efbf-4080-bd32-1b76006394a8.htseq.counts.gz c789f723-e543-4b62-98b4-444e5079fa71.htseq.counts.gz
43b292be-5d63-4523-a43f-666d20039208.htseq.counts.gz c789f723-e543-4b62-98b4-444e5079fa71.htseq.counts.gz
df959336-a712-47a9-9042-b2b6bc01e813.htseq.counts.gz c789f723-e543-4b62-98b4-444e5079fa71.htseq.counts.gz
b57f69a6-91ca-4131-a654-5495f6dd1131.htseq.counts.gz c789f723-e543-4b62-98b4-444e5079fa71.htseq.counts.gz
bf9c448b-bdc9-4f74-b13a-374e6add7939.htseq.counts.gz c789f723-e543-4b62-98b4-444e5079fa71.htseq.counts.gz
2c1aca0d-6c50-4f1e-9edf-fec9ec32da69.htseq.counts.gz c789f723-e543-4b62-98b4-444e5079fa71.htseq.counts.gz
bd8839a3-4dcb-4688-b051-6f09c18d7fc7.htseq.counts.gz c789f723-e543-4b62-98b4-444e5079fa71.htseq.counts.gz
65ec87f8-ca6b-4b47-9700-ef1b67589dec.htseq.counts.gz c789f723-e543-4b62-98b4-444e5079fa71.htseq.counts.gz
bda1a9a4-a14f-4463-81d2-a4fcca65d6f1.htseq.counts.gz c789f723-e543-4b62-98b4-444e5079fa71

file_name de808f1a-676c-4bda-afcb-bf61577324f7.htseq.counts.gz
1cbb39ca-efbf-4080-bd32-1b76006394a8.htseq.counts.gz de808f1a-676c-4bda-afcb-bf61577324f7.htseq.counts.gz
43b292be-5d63-4523-a43f-666d20039208.htseq.counts.gz de808f1a-676c-4bda-afcb-bf61577324f7.htseq.counts.gz
df959336-a712-47a9-9042-b2b6bc01e813.htseq.counts.gz de808f1a-676c-4bda-afcb-bf61577324f7.htseq.counts.gz
b57f69a6-91ca-4131-a654-5495f6dd1131.htseq.counts.gz de808f1a-676c-4bda-afcb-bf61577324f7.htseq.counts.gz
bf9c448b-bdc9-4f74-b13a-374e6add7939.htseq.counts.gz de808f1a-676c-4bda-afcb-bf61577324f7.htseq.counts.gz
2c1aca0d-6c50-4f1e-9edf-fec9ec32da69.htseq.counts.gz de808f1a-676c-4bda-afcb-bf61577324f7.htseq.counts.gz
bd8839a3-4dcb-4688-b051-6f09c18d7fc7.htseq.counts.gz de808f1a-676c-4bda-afcb-bf61577324f7.htseq.counts.gz
65ec87f8-ca6b-4b47-9700-ef1b67589dec.htseq.counts.gz de808f1a-676c-4bda-afcb-bf61577324f7.htseq.counts.gz
bda1a9a4-a14f-4463-81d2-a4fcca65d6f1.htseq.counts.gz de808f1a-676c-4bda-afcb-bf61577324f7

464cbc87-fa59-4fc9-8617-b697f3f29b02.htseq.counts.gz d60b1d68-c859-43af-b9bd-1a7dae06c804.htseq.counts.gz
32a115cf-570f-4ad9-a123-8e1970062f51.htseq.counts.gz d60b1d68-c859-43af-b9bd-1a7dae06c804.htseq.counts.gz
e86b6f79-cd01-4530-b315-c56e2fb0293f.htseq.counts.gz d60b1d68-c859-43af-b9bd-1a7dae06c804.htseq.counts.gz
02a81bc3-4672-4d2f-808d-d27f4b63bf85.htseq.counts.gz d60b1d68-c859-43af-b9bd-1a7dae06c804.htseq.counts.gz
03109baa-936f-49ae-acb3-2d00ac03c7ab.htseq.counts.gz d60b1d68-c859-43af-b9bd-1a7dae06c804.htseq.counts.gz
161caa2d-589a-4834-9073-3824031da5e6.htseq.counts.gz d60b1d68-c859-43af-b9bd-1a7dae06c804.htseq.counts.gz
5a58850f-397c-4786-8141-2d57dd974673.htseq.counts.gz d60b1d68-c859-43af-b9bd-1a7dae06c804.htseq.counts.gz
eda58f69-4a60-4072-aa1c-541a90d2d80b.htseq.counts.gz d60b1d68-c859-43af-b9bd-1a7dae06c804.htseq.counts.gz
3381f3a8-9040-4ea7-bb84-76a043f5d6b0.htseq.counts.gz d60b1d68-c859-43af-b9bd-1a7dae06c804.htseq.counts.gz
bbf586b3-2d8d-4a50-b5b1-7d0c75b6570d.htseq.cou

...

62f84581-4c7d-4c8e-835c-9304bcec3106.htseq.counts.gz f36f74a9-97b4-4913-86e2-56ebaf55f006.htseq.counts.gz
1e36b1ce-5849-48d5-8b9a-8102d833efc5.htseq.counts.gz f36f74a9-97b4-4913-86e2-56ebaf55f006.htseq.counts.gz
c12e6882-8bb8-448f-917c-dc9844fca4be.htseq.counts.gz f36f74a9-97b4-4913-86e2-56ebaf55f006.htseq.counts.gz
09977485-e1d1-41b1-b2b8-7ab0ad6df0a0.htseq.counts.gz f36f74a9-97b4-4913-86e2-56ebaf55f006.htseq.counts.gz
23fb4808-3b58-41b9-9c15-0ef53cafb713.htseq.counts.gz f36f74a9-97b4-4913-86e2-56ebaf55f006.htseq.counts.gz
0c3d283b-1ef5-459e-b312-c97ea0b6d793.htseq.counts.gz f36f74a9-97b4-4913-86e2-56ebaf55f006.htseq.counts.gz
6e933523-30a6-46a8-833d-7379b88c1c14.htseq.counts.gz f36f74a9-97b4-4913-86e2-56ebaf55f006.htseq.counts.gz
c14f98e2-8e9b-49f4-a244-3d06c6cb7126.htseq.counts.gz f36f74a9-97b4-4913-86e2-56ebaf55f006.htseq.counts.gz
9e97f017-6855-4d68-bd2a-c4da3ccb1d57.htseq.counts.gz f36f74a9-97b4-4913-86e2-56ebaf55f006.htseq.counts.gz
e82223ad-33c0-47fd-b0f2-982da64d0cb2.htseq.cou

a6cd69d5-ea54-4183-aac5-2b6efb311db3.htseq.counts.gz 7b2cd944-1cd6-4107-8aa6-11913ee32257.htseq.counts.gz
fd40f0e5-0825-409b-8a4c-86da09f0a84c.htseq.counts.gz 7b2cd944-1cd6-4107-8aa6-11913ee32257.htseq.counts.gz
1d975006-4e14-4f2c-be5f-e09f58790d62.htseq.counts.gz 7b2cd944-1cd6-4107-8aa6-11913ee32257.htseq.counts.gz
97503aa9-93a5-4ff6-bb19-203ea235c2bf.htseq.counts.gz 7b2cd944-1cd6-4107-8aa6-11913ee32257.htseq.counts.gz
b49d2b1e-310f-4278-a7c2-5ea95c2c7c43.htseq.counts.gz 7b2cd944-1cd6-4107-8aa6-11913ee32257.htseq.counts.gz
d60b1d68-c859-43af-b9bd-1a7dae06c804.htseq.counts.gz 7b2cd944-1cd6-4107-8aa6-11913ee32257.htseq.counts.gz
2bcd2efd-4fd6-40ee-86a4-867ae82711b0.htseq.counts.gz 7b2cd944-1cd6-4107-8aa6-11913ee32257.htseq.counts.gz
d5b4d77f-feb9-4a79-84a0-22849eb18112.htseq.counts.gz 7b2cd944-1cd6-4107-8aa6-11913ee32257.htseq.counts.gz
52e5f11b-9a73-4535-9e4d-720fe8feca0a.htseq.counts.gz 7b2cd944-1cd6-4107-8aa6-11913ee32257.htseq.counts.gz
96f97dd0-3c7a-4a8c-a165-9ab14c9fcef6.htseq.cou

file_name c49cf84f-cb19-4e11-9a71-4825bc906382.htseq.counts.gz
1cbb39ca-efbf-4080-bd32-1b76006394a8.htseq.counts.gz c49cf84f-cb19-4e11-9a71-4825bc906382.htseq.counts.gz
43b292be-5d63-4523-a43f-666d20039208.htseq.counts.gz c49cf84f-cb19-4e11-9a71-4825bc906382.htseq.counts.gz
df959336-a712-47a9-9042-b2b6bc01e813.htseq.counts.gz c49cf84f-cb19-4e11-9a71-4825bc906382.htseq.counts.gz
b57f69a6-91ca-4131-a654-5495f6dd1131.htseq.counts.gz c49cf84f-cb19-4e11-9a71-4825bc906382.htseq.counts.gz
bf9c448b-bdc9-4f74-b13a-374e6add7939.htseq.counts.gz c49cf84f-cb19-4e11-9a71-4825bc906382.htseq.counts.gz
2c1aca0d-6c50-4f1e-9edf-fec9ec32da69.htseq.counts.gz c49cf84f-cb19-4e11-9a71-4825bc906382.htseq.counts.gz
bd8839a3-4dcb-4688-b051-6f09c18d7fc7.htseq.counts.gz c49cf84f-cb19-4e11-9a71-4825bc906382.htseq.counts.gz
65ec87f8-ca6b-4b47-9700-ef1b67589dec.htseq.counts.gz c49cf84f-cb19-4e11-9a71-4825bc906382.htseq.counts.gz
bda1a9a4-a14f-4463-81d2-a4fcca65d6f1.htseq.counts.gz c49cf84f-cb19-4e11-9a71-4825bc906382

e7675043-6679-4179-af84-dc401102aa8c.htseq.counts.gz e4b288d5-b877-42fb-a525-8ed6424338db.htseq.counts.gz
c789f723-e543-4b62-98b4-444e5079fa71.htseq.counts.gz e4b288d5-b877-42fb-a525-8ed6424338db.htseq.counts.gz
1b843dbb-5ef0-47ca-9783-dbeb94aa6df3.htseq.counts.gz e4b288d5-b877-42fb-a525-8ed6424338db.htseq.counts.gz
9e189255-33ba-4802-934b-dd8a7898ef0a.htseq.counts.gz e4b288d5-b877-42fb-a525-8ed6424338db.htseq.counts.gz
c264fe3b-482b-44ec-83a4-73df565663ff.htseq.counts.gz e4b288d5-b877-42fb-a525-8ed6424338db.htseq.counts.gz
baf57a44-5eb7-4fdb-bbf6-ee006a122360.htseq.counts.gz e4b288d5-b877-42fb-a525-8ed6424338db.htseq.counts.gz
f96e90ca-9211-420e-a75f-5f30b67d2d95.htseq.counts.gz e4b288d5-b877-42fb-a525-8ed6424338db.htseq.counts.gz
762034f5-879b-4e74-a75d-064aee08c500.htseq.counts.gz e4b288d5-b877-42fb-a525-8ed6424338db.htseq.counts.gz
86679663-dfc5-46ad-8cf9-c7954c4b339b.htseq.counts.gz e4b288d5-b877-42fb-a525-8ed6424338db.htseq.counts.gz
d314696b-bbb9-425a-8a3a-5d32ec57af17.htseq.cou

9c52ed00-325f-4664-8873-327bcaa5ea74.htseq.counts.gz 95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz
171f6436-748a-4072-9ef6-e90d8cd9b465.htseq.counts.gz 95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz
f1fcd165-ddbc-44b8-b062-ae5199c53cba.htseq.counts.gz 95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz
7261b656-c79c-4581-a503-15b653e2b5d2.htseq.counts.gz 95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz
9daee176-d523-4450-8ac8-0f69d028399e.htseq.counts.gz 95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz
586cc5be-d91f-47ea-9d1e-c3b65b2af33f.htseq.counts.gz 95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz
fa73bdce-67fb-42aa-883f-635f0e7bcdc6.htseq.counts.gz 95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz
d47d93d3-64e9-40a3-a941-33432fb31a1a.htseq.counts.gz 95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz
f1038bc1-954d-4476-8abe-85093c206a5d.htseq.counts.gz 95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz
8bc10161-4cca-46c5-985e-e07349484ec2.htseq.cou